## TF IDF

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [11]:
df=pd.read_csv('C:\\Users\\Hrishi\\Desktop\\projects\\customer_support_analyzer\\data\\twcs.csv')
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [4]:
df.describe()

,tweet_id,in_response_to_tweet_id
count,2.811774e+06,2.017439e+06
mean,1.504565e+06,1.463141e+06
std,8.616450e+05,8.665730e+05
min,1.000000e+00,1.000000e+00
25%,7.601652e+05,7.155105e+05
50%,1.507772e+06,1.439805e+06
75%,2.253296e+06,2.220646e+06
max,2.987950e+06,2.987950e+06


In [5]:
df.shape

(2811774, 7)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811774 entries, 0 to 2811773
Data columns (total 7 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tweet_id                 int64  
 1   author_id                object 
 2   inbound                  bool   
 3   created_at               object 
 4   text                     object 
 5   response_tweet_id        object 
 6   in_response_to_tweet_id  float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 131.4+ MB


In [7]:
df=df[df['inbound']==True]
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0


In [8]:
df.shape

(1537843, 7)

In [12]:
train=df[['text']]
train.head()

,text
0,@115712 I understand. I would like to assist y...
1,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...
3,@115712 Please send us a Private Message so th...
4,@sprintcare I did.


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1537843 entries, 1 to 2811773
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   text    1537843 non-null  object
dtypes: object(1)
memory usage: 23.5+ MB


In [13]:
import re

In [14]:
def clean_txt(text):
    text=text.lower()
    text=re.sub(r"http\S+","",text)
    text=re.sub(r"@\w+","",text)
    text=re.sub(r"[^a-z\s]","",text)
    text=re.sub(r"\s+"," ",text).strip()
    return text

In [15]:
train["cleaned_text"]=train['text'].astype(str).apply(clean_txt)
train.head()

C:\Users\Hrishi\AppData\Local\Temp\ipykernel_22928\3574467968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["cleaned_text"]=train['text'].astype(str).apply(clean_txt)


,text,cleaned_text
0,@115712 I understand. I would like to assist y...,i understand i would like to assist you we wou...
1,@sprintcare and how do you propose we do that,and how do you propose we do that
2,@sprintcare I have sent several private messag...,i have sent several private messages and no on...
3,@115712 Please send us a Private Message so th...,please send us a private message so that we ca...
4,@sprintcare I did.,i did


In [16]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
sia=SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Hrishi\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [17]:
def get_sentiment(text):
    score=sia.polarity_scores(text)["compound"]
    if score<=-0.4:
        return "negative"
    elif score>=0.4:
        return "positive"
    else:
        return "neutral"
    
train['sentiment']=train['cleaned_text'].apply(get_sentiment)
train['sentiment'].value_counts()

C:\Users\Hrishi\AppData\Local\Temp\ipykernel_22928\2369130686.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['sentiment']=train['cleaned_text'].apply(get_sentiment)


sentiment
neutral     1516445
positive     955936
negative     339393
Name: count, dtype: int64

In [18]:
x=train["cleaned_text"]
y=train["sentiment"]

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_features=20000,
                      ngram_range=(1,2),
                      min_df=5,)
x_train_tfidf=tfidf.fit_transform(x_train)
x_test_tfidf=tfidf.transform(x_test)

In [19]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(max_iter=1000,n_jobs=-1)
clf.fit(x_train_tfidf,y_train)

from sklearn.metrics import classification_report
y_pred=clf.predict(x_test_tfidf)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

    negative       0.85      0.75      0.80     55843
     neutral       0.87      0.93      0.90    183062
    positive       0.88      0.80      0.84     68664

    accuracy                           0.87    307569
   macro avg       0.86      0.83      0.84    307569
weighted avg       0.87      0.87      0.87    307569



## LSTM MODEL

In [19]:
train_lstm=train.sample(200_000,random_state=42)

In [22]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train_lstm["label"]=le.fit_transform(train_lstm["sentiment"])
train_lstm["label"].value_counts()

label
1    108013
2     68033
0     23954
Name: count, dtype: int64

In [24]:
x=train_lstm["cleaned_text"]
y=train_lstm["label"]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)

NameError: name 'train_test_split' is not defined

In [23]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer=Tokenizer(num_words=20000,oov_token="<OOV>")
tokenizer.fit_on_texts(x_train)
x_train_seq=tokenizer.texts_to_sequences(x_train)
x_test_seq=tokenizer.texts_to_sequences(x_test)
x_train_pad=pad_sequences(x_train_seq,maxlen=50,padding="post")
x_test_pad=pad_sequences(x_test_seq,maxlen=50,padding="post")

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout,Bidirectional
from tensorflow.keras.layers import Input
model=Sequential([
    Input(shape=(50,)),
    Embedding(input_dim=30000,output_dim=128,input_length=50),
    Bidirectional(LSTM(32)),
    Dropout(0.3),
    Dense(3,activation="softmax"),
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
model.summary()

c:\Users\Hrishi\Desktop\projects\customer_support_analyzer\customer\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 50, 128)        │     3,840,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,881,411 (14.81 MB)

 Trainable params: 3,881,411 (14.81 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
history=model.fit(x_train_pad,y_train,
                  validation_split=0.1,
                  epochs=3,
                  batch_size=512)

Epoch 1/3
282/282 ━━━━━━━━━━━━━━━━━━━━ 36s 112ms/step - accuracy: 0.7390 - loss: 0.6195 - val_accuracy: 0.8470 - val_loss: 0.3874
Epoch 2/3
282/282 ━━━━━━━━━━━━━━━━━━━━ 32s 113ms/step - accuracy: 0.8772 - loss: 0.3299 - val_accuracy: 0.8762 - val_loss: 0.3207
Epoch 3/3
282/282 ━━━━━━━━━━━━━━━━━━━━ 31s 111ms/step - accuracy: 0.9102 - loss: 0.2510 - val_accuracy: 0.8921 - val_loss: 0.2828


In [26]:
from sklearn.metrics import classification_report
y_pred=np.argmax(model.predict(x_test_pad),axis=1)
print(classification_report(y_test,y_pred, target_names=le.classes_))

1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step
              precision    recall  f1-score   support

    negative       0.84      0.85      0.84      7281
     neutral       0.92      0.91      0.91     23762
    positive       0.87      0.89      0.88      8957

    accuracy                           0.89     40000
   macro avg       0.88      0.88      0.88     40000
weighted avg       0.89      0.89      0.89     40000



## BERT MODEL

In [4]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import torch

c:\Users\Hrishi\Desktop\projects\customer_support_analyzer\customer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer=AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch['cleaned_text'], padding="max_length", truncation=True, max_length=64)

In [25]:
from sklearn.model_selection import train_test_split

In [26]:

train_df, test_df = train_lstm[["cleaned_text", "label"]].pipe(
    lambda df: train_test_split(
        df,
        test_size=0.2,
        random_state=42,
        stratify=df["label"]
    )
)

In [27]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
test_ds  = Dataset.from_pandas(test_df.reset_index(drop=True))


In [28]:
tokenized_train = train_ds.map(tokenize, batched=True)
tokenized_test  = test_ds.map(tokenize, batched=True)

tokenized_train = tokenized_train.remove_columns(["cleaned_text"])
tokenized_test  = tokenized_test.remove_columns(["cleaned_text"])

tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_test  = tokenized_test.rename_column("label", "labels")

tokenized_train.set_format("torch")
tokenized_test.set_format("torch")


Map: 100%|██████████| 40000/40000 [00:02<00:00, 16627.07 examples/s]


In [29]:
from transformers import AutoModelForSequenceClassification

In [31]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    # use_safetensors=False,
    num_labels=3
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
training_args = TrainingArguments(
    output_dir="./bert_results",
    eval_strategy="epoch", 
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    fp16=True,                     
    weight_decay=0.01,
    logging_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="f1_negative" 
)


In [34]:
from sklearn.metrics import f1_score,accuracy_score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds=logits.argmax(axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_negative": f1_score(labels, preds, labels=[0],average="macro")
    }

In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


True
1
NVIDIA GeForce RTX 3050 Laptop GPU


In [1]:
import torch
torch.cuda.is_available()


True

In [35]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


C:\Users\Hrishi\AppData\Local\Temp\ipykernel_22928\1498825776.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Negative
1,0.149100,0.142514,0.944875,0.904676
2,0.104500,0.107213,0.964250,0.929633


TrainOutput(global_step=10000, training_loss=0.1772499584197998, metrics={'train_runtime': 1170.148, 'train_samples_per_second': 273.47, 'train_steps_per_second': 8.546, 'total_flos': 5298790440960000.0, 'train_loss': 0.1772499584197998, 'epoch': 2.0})

In [36]:
trainer.save_model("artifacts/bert_sentiment_model")
tokenizer.save_pretrained("artifacts/bert_sentiment_model")


('artifacts/bert_sentiment_model\\tokenizer_config.json',
 'artifacts/bert_sentiment_model\\special_tokens_map.json',
 'artifacts/bert_sentiment_model\\vocab.txt',
 'artifacts/bert_sentiment_model\\added_tokens.json',
 'artifacts/bert_sentiment_model\\tokenizer.json')